In [36]:
from typing import List, Tuple, Dict, Any, Optional
from multiprocessing import Pool
from pandarallel import pandarallel
import os
import re
import sys
import json
from pathlib import Path

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import pandas as pd
from tqdm.auto import tqdm
tqdm.pandas()
pandarallel.initialize(progress_bar=True)

INFO: Pandarallel will run on 24 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [37]:
from qiskit import QuantumCircuit
from qiskit.circuit.library.standard_gates import C3SXGate, U2Gate, CU3Gate

qc = QuantumCircuit(11)
qc.h(0)
qc.h(0)
qc.h(1)
qc.cx(0, 1)
qc.ry(4.237744763533372, 2)
qc.h(4)
# qc.cu3(3.744741759224157, 4.84182524491286, 2.9687539868895922, 10, 9)
qc.append(CU3Gate(3.744741759224157, 4.84182524491286,
          2.9687539868895922), [10, 9])
qc.cswap(10, 6, 8)
qc.x(10)
qc.crx(0.33702429296044073, 5, 3)
qc.x(3)
qc.crx(4.595031139687376, 7, 3)
qc.append(C3SXGate(), [5, 4, 1, 7])
qc.ry(5.669376893308966, 4)
qc.cp(3.8559792421298966, 6, 9)
qc.swap(3, 9)
qc.t(1)
qc.append(U2Gate(4.031122877773966, 0.5023290879741836), [0])
qc.rz(4.700807149748632, 1)

qc.draw()

┌───┐                 ┌───┐                »
 q_0: ───────────┤ H ├─────────────────┤ H ├──────────■─────»
                 ├───┤                 └───┘        ┌─┴─┐   »
 q_1: ───────────┤ H ├──────────────────────────────┤ X ├───»
             ┌───┴───┴────┐                         └───┘   »
 q_2: ───────┤ Ry(4.2377) ├─────────────────────────────────»
             └────────────┘       ┌─────────────┐   ┌───┐   »
 q_3: ────────────────────────────┤ Rx(0.33702) ├───┤ X ├───»
                 ┌───┐            └──────┬──────┘   └───┘   »
 q_4: ───────────┤ H ├───────────────────┼──────────────────»
                 └───┘                   │                  »
 q_5: ───────────────────────────────────■──────────────────»
                                                            »
 q_6: ───────────────────────────────────X────────■─────────»
                                         │        │         »
 q_7: ───────────────────────────────────┼────────┼─────────»
                                         │        │         »
 q_8: ───────────────────────────────────X────────┼─────────»
      ┌──────────────────────────┐       │        │P(3.856) »
 q_9: ┤ U3(3.7447,4.8418,2.9688) ├───────┼────────■─────────»
      └────────────┬─────────────┘       │          ┌───┐   »
q_10: ─────────────■─────────────────────■──────────┤ X ├───»
                                                    └───┘   »
«      ┌────────────────────┐                         
« q_0: ┤ U2(4.0311,0.50233) ├─────────────────────────
«      └────────────────────┘      ┌───┐┌────────────┐
« q_1: ────────────────────────■───┤ T ├┤ Rz(4.7008) ├
«                              │   └───┘└────────────┘
« q_2: ────────────────────────┼──────────────────────
«          ┌───────────┐       │                      
« q_3: ────┤ Rx(4.595) ├───────┼─────X────────────────
«          └─────┬─────┘       │     │  ┌────────────┐
« q_4: ──────────┼─────────────■─────┼──┤ Ry(5.6694) ├
«                │             │     │  └────────────┘
« q_5: ──────────┼─────────────■─────┼────────────────
«                │             │     │                
« q_6: ──────────┼─────────────┼─────┼────────────────
«                │           ┌─┴──┐  │                
« q_7: ──────────■───────────┤ Sx ├──┼────────────────
«                            └────┘  │                
« q_8: ──────────────────────────────┼────────────────
«                                    │                
« q_9: ──────────────────────────────X────────────────
«                                                     
«q_10: ───────────────────────────────────────────────
«

In [38]:
TEST_PROGRAM = "../program_bank/v037/2025_02_20__17_45/0000006_de436c.qasm"

# content of file
print(Path(TEST_PROGRAM).read_text())

OPENQASM 2.0;
include "qelib1.inc";
qreg q[11];
ry(4.237744763533372) q[2];
h q[4];
cu3(3.744741759224157,4.84182524491286,2.9687539868895922) q[10],q[9];
cswap q[10],q[6],q[8];
x q[10];
crx(0.33702429296044073) q[5],q[3];
x q[3];
crx(4.595031139687376) q[7],q[3];
c3sqrtx q[5],q[4],q[1],q[7];
ry(5.669376893308966) q[4];
cp(3.8559792421298966) q[6],q[9];
swap q[3],q[9];
t q[1];
u2(4.031122877773966,0.5023290879741836) q[9];
rz(4.700807149748632) q[1];


In [39]:
from bqskit import Circuit

qc_bqskit_qasm = Circuit.from_file(TEST_PROGRAM)
qc_bqskit_qasm

Circuit(11)
	[None, None, RYGate([4.237744763533372])@(2,), CRXGate([0.33702429296044073])@(5, 3), HGate@(4,), CRXGate([0.33702429296044073])@(5, 3), None, None, None, Controlled(U3Gate)([3.744741759224157, 4.84182524491286, 2.9687539868895922])@(10, 9), Controlled(U3Gate)([3.744741759224157, 4.84182524491286, 2.9687539868895922])@(10, 9)]
	[None, None, None, XGate@(3,), None, None, Controlled(SwapGate)@(10, 6, 8), None, Controlled(SwapGate)@(10, 6, 8), None, Controlled(SwapGate)@(10, 6, 8)]
	[None, None, None, CRXGate([4.595031139687376])@(7, 3), None, None, CPGate([3.8559792421298966])@(6, 9), CRXGate([4.595031139687376])@(7, 3), None, CPGate([3.8559792421298966])@(6, 9), XGate@(10,)]
	[None, Controlled(SqrtXGate)@(5, 4, 1, 7), None, SwapGate@(3, 9), Controlled(SqrtXGate)@(5, 4, 1, 7), Controlled(SqrtXGate)@(5, 4, 1, 7), None, Controlled(SqrtXGate)@(5, 4, 1, 7), None, SwapGate@(3, 9), None]
	[None, TGate@(1,), None, None, RYGate([5.669376893308966])@(4,), None, None, None, None, U2Ga

In [40]:
from qiskit.qasm2 import load, LEGACY_CUSTOM_INSTRUCTIONS
from bqskit.ext import qiskit_to_bqskit
qc_via_qiskit = load(
    TEST_PROGRAM, custom_instructions=LEGACY_CUSTOM_INSTRUCTIONS)
qc_bqskit_via_qiskit = qiskit_to_bqskit(qc_via_qiskit)
qc_bqskit_via_qiskit

Circuit(11)
	[None, None, RYGate([4.237744763533372])@(2,), CRXGate([0.33702429296044073])@(5, 3), HGate@(4,), CRXGate([0.33702429296044073])@(5, 3), None, None, None, Controlled(U3Gate)([3.744741759224157, 4.84182524491286, 2.9687539868895922])@(10, 9), Controlled(U3Gate)([3.744741759224157, 4.84182524491286, 2.9687539868895922])@(10, 9)]
	[None, None, None, XGate@(3,), None, None, Controlled(SwapGate)@(10, 6, 8), None, Controlled(SwapGate)@(10, 6, 8), None, Controlled(SwapGate)@(10, 6, 8)]
	[None, None, None, CRXGate([4.595031139687376])@(7, 3), None, None, CPGate([3.8559792421298966])@(6, 9), CRXGate([4.595031139687376])@(7, 3), None, CPGate([3.8559792421298966])@(6, 9), XGate@(10,)]
	[None, Controlled(SqrtXGate)@(5, 4, 1, 7), None, SwapGate@(3, 9), Controlled(SqrtXGate)@(5, 4, 1, 7), Controlled(SqrtXGate)@(5, 4, 1, 7), None, Controlled(SqrtXGate)@(5, 4, 1, 7), None, SwapGate@(3, 9), None]
	[None, TGate@(1,), None, None, RYGate([5.669376893308966])@(4,), None, None, None, None, U2Ga

In [41]:
qc_bqskit_qasm == qc_bqskit_via_qiskit

True

In [50]:
from bqskit import compile
from bqskit.compiler import Compiler, Workflow
from bqskit.passes import (
    AutoRebase2QuditGatePass,
    QuickPartitioner,
    ScanningGateRemovalPass,
    UnfoldPass,
    ExhaustiveGateRemovalPass
)

from qiskit import QuantumCircuit
from qiskit.circuit.library.standard_gates import C3SXGate, U2Gate, CU3Gate

qc = QuantumCircuit(10)
qc.h(0)
qc.h(0)
qc.h(1)
qc.cx(0, 1)
# qc.ry(4.237744763533372, 2)
qc.h(4)
# qc.cu3(3.744741759224157, 4.84182524491286, 2.9687539868895922, 3, 9)
qc.append(CU3Gate(3.744741759224157, 4.84182524491286,
                  2.9687539868895922), [3, 9])
qc.cswap(9, 6, 8)
qc.x(9)
qc.crx(0.33702429296044073, 5, 3)
qc.x(3)
qc.crx(4.595031139687376, 7, 3)
# qc.append(C3SXGate(), [5, 4, 1, 7])
qc.ry(5.669376893308966, 4)
# qc.cp(3.8559792421298966, 6, 9)
qc.swap(3, 9)
qc.t(1)
# qc.append(U2Gate(4.031122877773966, 0.5023290879741836), [0])
qc.rz(4.700807149748632, 1)
print(qc.draw())

qc_bqskit = qiskit_to_bqskit(qc)

workflow = Workflow(
    passes=[
        QuickPartitioner(),
        ScanningGateRemovalPass(),
        # UnfoldPass(),
        # ExhaustiveGateRemovalPass(),
        # AutoRebase2QuditGatePass()
    ]
)

# with Compiler() as compiler:
# compiled_circuit = compiler.compile(qc_bqskit, workflow)
compiled_circuit = compile(qc_bqskit, optimization_level=1)


compiled_circuit

     ┌───┐           ┌───┐                                         »
q_0: ┤ H ├───────────┤ H ├───────────────────■─────────────────────»
     ├───┤           └───┘                 ┌─┴─┐         ┌───┐     »
q_1: ┤ H ├─────────────────────────────────┤ X ├─────────┤ T ├─────»
     └───┘                                 └───┘         └───┘     »
q_2: ──────────────────────────────────────────────────────────────»
                                      ┌─────────────┐    ┌───┐     »
q_3: ──────────────────■──────────────┤ Rx(0.33702) ├────┤ X ├─────»
     ┌───┐             │              └──────┬──────┘┌───┴───┴────┐»
q_4: ┤ H ├─────────────┼─────────────────────┼───────┤ Ry(5.6694) ├»
     └───┘             │                     │       └────────────┘»
q_5: ──────────────────┼─────────────────────■─────────────────────»
                       │                                           »
q_6: ──────────────────┼─────────────────────X─────────────────────»
                       │          

Circuit(10)
	[U3Gate([2.4233767764017036e-16, 0.0, -1.2246467991473532e-16])@(0,), U3Gate([1.5707963267948966, 0.0, 3.141592653589793])@(1,), None, U3Gate([1.5707963268098035, 4.140765103411197, -0.05978914865507701])@(3,), U3Gate([0.9569879129242762, 0.0, -3.141592653589793])@(4,), U3Gate([1.0235549993480946, 1.5707963288293079, 0.3531517454650558])@(5,), U3Gate([2.082697361439754, -2.5688443637276372, 3.4541331349953697])@(6,), U3Gate([4.7123889802481935, 3.141592652759739, 1.8445831942908695])@(7,), U3Gate([2.393056761213169, 0.8034369711929426, -0.608633790498627])@(8,), U3Gate([2.0661407400638314, 1.3294084945394518, 2.0885650787783963])@(9,)]
	[CNOTGate@(0, 1), CNOTGate@(0, 1), None, CNOTGate@(3, 9), None, None, CNOTGate@(6, 8), None, CNOTGate@(6, 8), CNOTGate@(3, 9)]
	[U3Gate([0.0, 0.0, 0.0])@(0,), U3Gate([0.0, 2.7431026565730403, 2.7431026565730403])@(1,), None, U3Gate([1.354579813324752, 3.6100260581121706, 0.5716238769825637])@(3,), None, None, U3Gate([0.7853981633974348, 1.5